In [1]:
!pip install lightgbm

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r"C:\Users\moham\Data_processing_First_[2].csv", low_memory=False)
df.head()

,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,diag_1,...,glyburide,insulin,change,diabetesMed,readmitted,service_utilization,Asian,Caucasian,Hispanic,Other
0,0,15,0,0,0,3,59,0,18,8,...,0,1,1,1,0,0,0,1,0,0
1,0,25,0,0,0,2,11,5,13,8,...,0,0,0,1,0,3,0,0,0,0
2,1,35,0,0,0,2,44,1,16,8,...,0,1,1,1,0,0,0,1,0,0
3,1,45,0,0,0,1,51,0,8,8,...,0,0,1,1,0,0,0,1,0,0
4,1,55,0,0,1,3,31,6,16,8,...,0,0,0,1,0,0,0,1,0,0


In [4]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, 
    AdaBoostClassifier, 
    GradientBoostingClassifier, 
    StackingClassifier, 
    BaggingClassifier 
)
from sklearn.model_selection import train_test_split, GridSearchCV, KFold,cross_val_score
from sklearn.metrics import confusion_matrix, classification_report,precision_score, recall_score, f1_score,roc_auc_score
from sklearn.pipeline import Pipeline
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.naive_bayes import GaussianNB
import lightgbm as ltb
from sklearn.metrics import roc_curve
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [5]:
# Drop rows with 'Unknown/Invalid' in any column
df = df.replace('Unknown/Invalid', np.nan)  # mark them as NaN
df = df.dropna()  # remove those rows

In [6]:
X = df.drop('readmitted', axis=1)
y = df['readmitted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (68461, 26)
X_test: (29341, 26)
y_train: (68461,)
y_test: (29341,)


In [7]:
scores = pd.DataFrame(columns=['Model_Name', 'Train_Accuracy', 'Test_Accuracy',
                               'Train_f1', 'Train_precision', 'Train_recall', 'Train_auc_roc',
                               'Test_f1', 'Test_precision', 'Test_recall', 'Test_auc_roc'])

scores

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc


In [8]:
# LOGISTICREGRESSION

In [9]:
# Pipeline setup

pipe = Pipeline((
("pt", PowerTransformer()),
("lr", LogisticRegression()),
))

# Fit the model

pipe.fit(X_train,y_train)

# Accuracy

print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)

# Cross-validation

score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)

# Predictions

pred=pipe.predict(X_test)
predx=pipe.predict(X_train)

# Test evaluation

print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)

# Train evaluation

print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)

# Save results in DataFrame

lr1 = pd.Series({'Model_Name': "Logistic Regression Base", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, lr1.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.8862564087582712
Testing Accuracy
0.8834736375719983
Cross validation score 0.8862564093012548
[[25922     0]
 [ 3419     0]]
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     25922
           1       0.00      0.00      0.00      3419

    accuracy                           0.88     29341
   macro avg       0.44      0.50      0.47     29341
weighted avg       0.78      0.88      0.83     29341

precision 0.0
recall 0.0
f1_score 0.0
roc auc_score 0.5
[[60674     0]
 [ 7787     0]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     60674
           1       0.00      0.00      0.00      7787

    accuracy                           0.89     68461
   macro avg       0.44      0.50      0.47     68461
weighted avg       0.79      0.89      0.83     68461

precision 0.0
recall 0.0
f1_score 0.0
roc_auc_score 0.5


,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5


In [10]:
pipe = Pipeline((
("pt", PowerTransformer()),
("lr", LogisticRegression(class_weight='balanced')),
))

pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)

score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)


add = pd.Series({'Model_Name': "Logistic Regression Balanced", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.5914316180014899
Testing Accuracy
0.5916635424832146
Cross validation score 0.591124865772944
[[15232 10690]
 [ 1291  2128]]
              precision    recall  f1-score   support

           0       0.92      0.59      0.72     25922
           1       0.17      0.62      0.26      3419

    accuracy                           0.59     29341
   macro avg       0.54      0.61      0.49     29341
weighted avg       0.83      0.59      0.66     29341

precision 0.16601653924169138
recall 0.6224042117578239
f1_score 0.26211738621666564
roc auc_score 0.6050065962731717
[[35670 25004]
 [ 2967  4820]]
              precision    recall  f1-score   support

           0       0.92      0.59      0.72     60674
           1       0.16      0.62      0.26      7787

    accuracy                           0.59     68461
   macro avg       0.54      0.60      0.49     68461
weighted avg       0.84      0.59      0.67     68461

precision 0.1616148068669528
recall 0.61898035186849

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007


In [11]:
# DECISIONN TREE

In [12]:
pipe = Pipeline((
("pt", PowerTransformer()),
("dt", DecisionTreeClassifier(random_state=10)),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)


add = pd.Series({'Model_Name': "Decision Tree", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
1.0
Testing Accuracy
0.7835111277734229
Cross validation score 0.7865646949607945
[[22421  3501]
 [ 2851   568]]
              precision    recall  f1-score   support

           0       0.89      0.86      0.88     25922
           1       0.14      0.17      0.15      3419

    accuracy                           0.78     29341
   macro avg       0.51      0.52      0.51     29341
weighted avg       0.80      0.78      0.79     29341

precision 0.13959203735561562
recall 0.16613044749926878
f1_score 0.1517094017094017
roc auc_score 0.5155357121378761
[[60674     0]
 [    0  7787]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60674
           1       1.00      1.00      1.00      7787

    accuracy                           1.00     68461
   macro avg       1.00      1.00      1.00     68461
weighted avg       1.00      1.00      1.00     68461

precision 1.0
recall 1.0
f1_score 1.0
roc_auc_score 1.0


,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536


In [13]:
pipe = Pipeline((
("pt", PowerTransformer()),
("dt", DecisionTreeClassifier(random_state=10,class_weight='balanced')),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "Decision Tree Balanced", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
1.0
Testing Accuracy
0.8033127705258852
Cross validation score 0.8029096519288768
[[23085  2837]
 [ 2934   485]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89     25922
           1       0.15      0.14      0.14      3419

    accuracy                           0.80     29341
   macro avg       0.52      0.52      0.52     29341
weighted avg       0.80      0.80      0.80     29341

precision 0.14599638771824203
recall 0.14185434337525593
f1_score 0.14389556445631213
roc auc_score 0.5162053138062916
[[60674     0]
 [    0  7787]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60674
           1       1.00      1.00      1.00      7787

    accuracy                           1.00     68461
   macro avg       1.00      1.00      1.00     68461
weighted avg       1.00      1.00      1.00     68461

precision 1.0
recall 1.0
f1_score 1.0
roc_auc_score 1.0


,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205


In [14]:
pipe = Pipeline((
("pt", PowerTransformer()),
("dt", DecisionTreeClassifier(random_state=10,class_weight='balanced',criterion = 'entropy',max_depth = 5)),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "Decision Tree Tuned", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.6281970757073371
Testing Accuracy
0.630244367949286
Cross validation score 0.5981360747388708
[[16537  9385]
 [ 1464  1955]]
              precision    recall  f1-score   support

           0       0.92      0.64      0.75     25922
           1       0.17      0.57      0.26      3419

    accuracy                           0.63     29341
   macro avg       0.55      0.60      0.51     29341
weighted avg       0.83      0.63      0.70     29341

precision 0.17239858906525574
recall 0.571804621234279
f1_score 0.26492309777085166
roc auc_score 0.6048784698641111
[[38486 22188]
 [ 3266  4521]]
              precision    recall  f1-score   support

           0       0.92      0.63      0.75     60674
           1       0.17      0.58      0.26      7787

    accuracy                           0.63     68461
   macro avg       0.55      0.61      0.51     68461
weighted avg       0.84      0.63      0.70     68461

precision 0.16926878580253846
recall 0.58058302298702

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878


In [15]:
#RANDOM_FOREST

In [16]:
pipe = Pipeline((
("pt", PowerTransformer()),
("dt", RandomForestClassifier(random_state=10)),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "RandomForest", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.9999853931435416
Testing Accuracy
0.8833373095668178
Cross validation score 0.8861541630127657
[[25910    12]
 [ 3411     8]]
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     25922
           1       0.40      0.00      0.00      3419

    accuracy                           0.88     29341
   macro avg       0.64      0.50      0.47     29341
weighted avg       0.83      0.88      0.83     29341

precision 0.4
recall 0.00233986545773618
f1_score 0.004652515266065717
roc auc_score 0.5009384691072339
[[60674     0]
 [    1  7786]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60674
           1       1.00      1.00      1.00      7787

    accuracy                           1.00     68461
   macro avg       1.00      1.00      1.00     68461
weighted avg       1.00      1.00      1.00     68461

precision 1.0
recall 0.9998715808398613
f1_score 0.99993578629679

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938


In [17]:
pipe = Pipeline((
("pt", PowerTransformer()),
("rf", RandomForestClassifier(random_state=10,class_weight='balanced')),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "Randoom Forest Balanced", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.9999853931435416
Testing Accuracy
0.8835758835758836
Cross validation score 0.8860811287304113
[[25918     4]
 [ 3412     7]]
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     25922
           1       0.64      0.00      0.00      3419

    accuracy                           0.88     29341
   macro avg       0.76      0.50      0.47     29341
weighted avg       0.85      0.88      0.83     29341

precision 0.6363636363636364
recall 0.0020473822755191578
f1_score 0.004081632653061225
roc auc_score 0.5009465365972149
[[60674     0]
 [    1  7786]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60674
           1       1.00      1.00      1.00      7787

    accuracy                           1.00     68461
   macro avg       1.00      1.00      1.00     68461
weighted avg       1.00      1.00      1.00     68461

precision 1.0
recall 0.9998715808398613
f1_score

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947


In [18]:
pipe = Pipeline((
("pt", PowerTransformer()),
("rf", RandomForestClassifier(random_state=10,n_estimators = 5,max_depth = 3,class_weight='balanced')),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "Random Forest Tuned", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.634258921137582
Testing Accuracy
0.635527078150029
Cross validation score 0.6322428957828399
[[16749  9173]
 [ 1521  1898]]
              precision    recall  f1-score   support

           0       0.92      0.65      0.76     25922
           1       0.17      0.56      0.26      3419

    accuracy                           0.64     29341
   macro avg       0.54      0.60      0.51     29341
weighted avg       0.83      0.64      0.70     29341

precision 0.17143889440881582
recall 0.5551330798479087
f1_score 0.2619737750172533
roc auc_score 0.6006318898197958
[[39046 21628]
 [ 3411  4376]]
              precision    recall  f1-score   support

           0       0.92      0.64      0.76     60674
           1       0.17      0.56      0.26      7787

    accuracy                           0.63     68461
   macro avg       0.54      0.60      0.51     68461
weighted avg       0.83      0.63      0.70     68461

precision 0.1682818027995693
recall 0.5619622447669192

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947
7,Random Forest Tuned,0.632243,0.635527,0.259004,0.168282,0.561962,0.60275,0.261974,0.171439,0.555133,0.600632


In [19]:
#XG Boost

In [20]:
pipe = Pipeline((
("pt", PowerTransformer()),
("dt", XGBClassifier()),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)
print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "XG Boost", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.8930193832985204
Testing Accuracy
0.8830646535564568
Cross validation score 0.8848541583339804
[[25868    54]
 [ 3377    42]]
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     25922
           1       0.44      0.01      0.02      3419

    accuracy                           0.88     29341
   macro avg       0.66      0.51      0.48     29341
weighted avg       0.83      0.88      0.83     29341

precision 0.4375
recall 0.012284293653114946
f1_score 0.023897581792318635
roc auc_score 0.5051005605292038
[[60659    15]
 [ 7309   478]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     60674
           1       0.97      0.06      0.12      7787

    accuracy                           0.89     68461
   macro avg       0.93      0.53      0.53     68461
weighted avg       0.90      0.89      0.85     68461

precision 0.9695740365111561
recall 0.061384358546295106
f1_s

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947
7,Random Forest Tuned,0.632243,0.635527,0.259004,0.168282,0.561962,0.60275,0.261974,0.171439,0.555133,0.600632
8,XG Boost,0.884854,0.883065,0.115459,0.969574,0.061384,0.530569,0.023898,0.4375,0.012284,0.505101


In [21]:
pipe = Pipeline((
("pt", PowerTransformer()),
("gb", XGBClassifier(scale_pos_weight=7.73)),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "XG Boost Balanced", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.7365799506288252
Testing Accuracy
0.6771752837326608
Cross validation score 0.6811615011105234
[[18237  7685]
 [ 1787  1632]]
              precision    recall  f1-score   support

           0       0.91      0.70      0.79     25922
           1       0.18      0.48      0.26      3419

    accuracy                           0.68     29341
   macro avg       0.54      0.59      0.53     29341
weighted avg       0.83      0.68      0.73     29341

precision 0.1751636792959107
recall 0.47733255337818076
f1_score 0.2562814070351759
roc auc_score 0.5904331156675643
[[44489 16185]
 [ 1849  5938]]
              precision    recall  f1-score   support

           0       0.96      0.73      0.83     60674
           1       0.27      0.76      0.40      7787

    accuracy                           0.74     68461
   macro avg       0.61      0.75      0.61     68461
weighted avg       0.88      0.74      0.78     68461

precision 0.26840844370112554
recall 0.7625529729035

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947
7,Random Forest Tuned,0.632243,0.635527,0.259004,0.168282,0.561962,0.60275,0.261974,0.171439,0.555133,0.600632
8,XG Boost,0.884854,0.883065,0.115459,0.969574,0.061384,0.530569,0.023898,0.4375,0.012284,0.505101
9,XG Boost Balanced,0.681162,0.677175,0.397058,0.268408,0.762553,0.7479,0.256281,0.175164,0.477333,0.590433


In [22]:
pipe = Pipeline((
("pt", PowerTransformer()),
("gb", XGBClassifier(learning_rate=.1,max_depth = 4, gamma = 8,scale_pos_weight=7.72)),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "XG Boost Tuned", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

Training Accuracy
0.627919545434627
Testing Accuracy
0.6242800177226406
Cross validation score 0.6196082113558197
[[16287  9635]
 [ 1389  2030]]
              precision    recall  f1-score   support

           0       0.92      0.63      0.75     25922
           1       0.17      0.59      0.27      3419

    accuracy                           0.62     29341
   macro avg       0.55      0.61      0.51     29341
weighted avg       0.83      0.62      0.69     29341

precision 0.17402486069438491
recall 0.5937408599005557
f1_score 0.26915937417130736
roc auc_score 0.6110244304132051
[[38135 22539]
 [ 2934  4853]]
              precision    recall  f1-score   support

           0       0.93      0.63      0.75     60674
           1       0.18      0.62      0.28      7787

    accuracy                           0.63     68461
   macro avg       0.55      0.63      0.51     68461
weighted avg       0.84      0.63      0.70     68461

precision 0.1771685163551402
recall 0.62321818415307

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947
7,Random Forest Tuned,0.632243,0.635527,0.259004,0.168282,0.561962,0.60275,0.261974,0.171439,0.555133,0.600632
8,XG Boost,0.884854,0.883065,0.115459,0.969574,0.061384,0.530569,0.023898,0.4375,0.012284,0.505101
9,XG Boost Balanced,0.681162,0.677175,0.397058,0.268408,0.762553,0.7479,0.256281,0.175164,0.477333,0.590433


In [23]:
#LightGBM

In [24]:
pipe = Pipeline((
("pt", PowerTransformer()),
("lg", LGBMClassifier()),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "LightGBM", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

  File "C:\Users\moham\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Number of positive: 7787, number of negative: 60674
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005406 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 68461, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.113744 -> initscore=-2.053060
[LightGBM] [Info] Start training from score -2.053060
Training Accuracy
0.8870597858634843
Testing Accuracy
0.8835418015745885
[LightGBM] [Info] Number of positive: 6229, number of negative: 48539
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003218 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 352
[LightGBM] [Info] Number of data points in

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947
7,Random Forest Tuned,0.632243,0.635527,0.259004,0.168282,0.561962,0.60275,0.261974,0.171439,0.555133,0.600632
8,XG Boost,0.884854,0.883065,0.115459,0.969574,0.061384,0.530569,0.023898,0.4375,0.012284,0.505101
9,XG Boost Balanced,0.681162,0.677175,0.397058,0.268408,0.762553,0.7479,0.256281,0.175164,0.477333,0.590433


In [25]:
pipe = Pipeline((
("pt", PowerTransformer()),
("lg", LGBMClassifier(class_weight='balanced')),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "LightGBM Balanced", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

[LightGBM] [Info] Number of positive: 7787, number of negative: 60674
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006092 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 68461, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training Accuracy
0.6486028541797519
Testing Accuracy
0.6256092157731502
[LightGBM] [Info] Number of positive: 6229, number of negative: 48539
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004434 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 352
[LightGBM] [Info] Number of data points in

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947
7,Random Forest Tuned,0.632243,0.635527,0.259004,0.168282,0.561962,0.60275,0.261974,0.171439,0.555133,0.600632
8,XG Boost,0.884854,0.883065,0.115459,0.969574,0.061384,0.530569,0.023898,0.4375,0.012284,0.505101
9,XG Boost Balanced,0.681162,0.677175,0.397058,0.268408,0.762553,0.7479,0.256281,0.175164,0.477333,0.590433


In [26]:
pipe = Pipeline((
("pt", PowerTransformer()),
("lg", LGBMClassifier(objective='binary',max_bin=7,num_leaves=7,max_depth=4,random_state=5,class_weight='balanced')),
))


pipe.fit(X_train,y_train)
print("Training Accuracy")
print(pipe.score(X_train,y_train))
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
te_acc=pipe.score(X_test,y_test)


score=cross_val_score(estimator=pipe,X=X_train,y=y_train, scoring= 'accuracy',cv=5)
print('Cross validation score',np.mean(score))
tr_acc=np.mean(score)


pred=pipe.predict(X_test)
predx=pipe.predict(X_train)


print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print('precision',precision_score(y_test,pred))
p=precision_score(y_test,pred)
print('recall', recall_score(y_test,pred))
r=recall_score(y_test,pred)
print('f1_score',f1_score(y_test, pred))
f=f1_score(y_test,pred)
print('roc auc_score', roc_auc_score(y_test, pred))
au=roc_auc_score(y_test,pred)


print(confusion_matrix(y_train,predx))
print(classification_report(y_train, predx))
print('precision', precision_score(y_train, predx))
tp=precision_score(y_train,predx)
print('recall', recall_score(y_train,predx))
tr=recall_score(y_train,predx)
print('f1_score', f1_score(y_train, predx))
tf=f1_score(y_train,predx)
print('roc_auc_score', roc_auc_score(y_train,predx))
tau=roc_auc_score(y_train, predx)



add = pd.Series({'Model_Name': "LightGBM Tuned", 'Train_Accuracy':tr_acc, 'Test_Accuracy': te_acc, 'Test_f1':f, 'Test_recall':r, 'Test_precision':p, 'Test_auc_roc':au,'Train_f1' : tf, 'Train_recall': tr, 'Train_precision': tp, 'Train_auc_roc':tau})

scores = pd.concat([scores, add.to_frame().T], ignore_index=True)


scores

[LightGBM] [Info] Number of positive: 7787, number of negative: 60674
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 121
[LightGBM] [Info] Number of data points in the train set: 68461, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training Accuracy
0.606126115598662
Testing Accuracy
0.6025357008963567
[LightGBM] [Info] Number of positive: 6229, number of negative: 48539
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 121
[LightGBM] [Info] Number of data points in 

,Model_Name,Train_Accuracy,Test_Accuracy,Train_f1,Train_precision,Train_recall,Train_auc_roc,Test_f1,Test_precision,Test_recall,Test_auc_roc
0,Logistic Regression Base,0.886256,0.883474,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5
1,Logistic Regression Balanced,0.591125,0.591664,0.256308,0.161615,0.61898,0.603438,0.262117,0.166017,0.622404,0.605007
2,Decision Tree,0.786565,0.783511,1.0,1.0,1.0,1.0,0.151709,0.139592,0.16613,0.515536
3,Decision Tree Balanced,0.80291,0.803313,1.0,1.0,1.0,1.0,0.143896,0.145996,0.141854,0.516205
4,Decision Tree Tuned,0.598136,0.630244,0.262117,0.169269,0.580583,0.607445,0.264923,0.172399,0.571805,0.604878
5,RandomForest,0.886154,0.883337,0.999936,1.0,0.999872,0.999936,0.004653,0.4,0.00234,0.500938
6,Randoom Forest Balanced,0.886081,0.883576,0.999936,1.0,0.999872,0.999936,0.004082,0.636364,0.002047,0.500947
7,Random Forest Tuned,0.632243,0.635527,0.259004,0.168282,0.561962,0.60275,0.261974,0.171439,0.555133,0.600632
8,XG Boost,0.884854,0.883065,0.115459,0.969574,0.061384,0.530569,0.023898,0.4375,0.012284,0.505101
9,XG Boost Balanced,0.681162,0.677175,0.397058,0.268408,0.762553,0.7479,0.256281,0.175164,0.477333,0.590433


In [54]:
!pip install joblib

In [56]:
import pickle


In [58]:
pickle.dump(pipe,open('model.pkl','wb'))

In [60]:
model1 = pickle.load(open('model.pkl','rb'))

In [62]:
print(model1.predict([[0,15,0,0,0,3,59,0,18,8,0,8,9,0,0,0,0,0,1,1,1,0,0,1,0,0
]]))

[0]


In [68]:
print(model1.predict([[0,45,0,1,0,7,60,0,15,8,0,0,8,0,0,0,0,0,1,1,1,1,0,1,0,0

]]))

[1]
